# 卷积神经网络LeNet

In [1]:
#coding=utf-8
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

import numpy as np
import sys
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

from torchvision.datasets import mnist # 导入 pytorch 内置的 mnist 数据

## Step1-使用MNIST数据集

In [4]:
# We transform them to Tensors of normalized range [-1, 1]
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                             ])

train_set = mnist.MNIST('../../../dataset', train=True, transform=transform, download=False) 
test_set = mnist.MNIST('../../../dataset', train=False, transform=transform, download=False)

from torch.utils.data import DataLoader
# 使用 pytorch 自带的 DataLoader 定义一个数据迭代器
trainloader = DataLoader(train_set, batch_size=64, shuffle=True)
testloader = DataLoader(test_set, batch_size=128, shuffle=False)

## Step2-定义卷积网络LeNet

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2) 
        self.pool  = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

## Step3-训练网络

In [ ]:
criterion = nn.CrossEntropyLoss() #叉熵损失函数
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #使用SGD（随机梯度下降）优化，学习率为0.001，动量为0.9

for epoch in range(3): # 遍历数据集
    
    running_loss = 0.0
    #enumerate(sequence, [start=0])，i序号，data是数据
    for i, data in enumerate(trainloader, 0): 
        # get the inputs
        inputs, labels = data   #data的结构是：[4x3x32x32的张量,长度4的张量]
        
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)  #把input数据从tensor转为variable
        
        # zero the parameter gradients
        optimizer.zero_grad() #将参数的grad值初始化为0
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels) #将output和labels使用叉熵计算损失
        loss.backward() #反向传播
        optimizer.step() #用SGD更新参数
        
        # 每100批数据打印一次平均loss值
        running_loss += loss.data[0]  #loss本身为Variable类型，所以要使用data获取其Tensor，因为其为标量，所以取0
        if i % 100 == 99: # 每100批打印一次
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 0.465
[1,   200] loss: 0.415
[1,   300] loss: 0.364
[1,   400] loss: 0.320
[1,   500] loss: 0.318
[1,   600] loss: 0.273
[1,   700] loss: 0.247
[1,   800] loss: 0.242
[1,   900] loss: 0.223
[2,   100] loss: 0.208
[2,   200] loss: 0.194
[2,   300] loss: 0.179
[2,   400] loss: 0.181
[2,   500] loss: 0.169
[2,   600] loss: 0.141
[2,   700] loss: 0.144
[2,   800] loss: 0.139
[2,   900] loss: 0.150
[3,   100] loss: 0.125
[3,   200] loss: 0.134
[3,   300] loss: 0.112
[3,   400] loss: 0.125
[3,   500] loss: 0.121
[3,   600] loss: 0.116
[3,   700] loss: 0.115
[3,   800] loss: 0.110
[3,   900] loss: 0.113


### 测试集正确率

In [8]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    #print outputs.data
    _, predicted = torch.max(outputs.data, 1)  #outputs.data是一个4x10张量，将每一行的最大的那一列的值和序号各自组成一个一维张量返回，第一个是值的张量，第二个是序号的张量。
    total += labels.size(0)
    correct += (predicted == labels).sum()   #两个一维张量逐行对比，相同的行记为1，不同的行记为0，再利用sum(),求总和，得到相同的个数。

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

Accuracy of the network on the test images: 86 %
